In [7]:
import time

In [8]:
from src.trash.simulation_tools import compute_flows, run_simulation
import src.trash.nernst_potentials as P

In [9]:
import src.trash.ionic_fluxes as i_flux
import src.trash.dep_functions as dep_funct

# Old Masha's code

## Some imports turned into plane code

In [10]:
import numpy as np

# mapping of chemicals to numerical indices used to index the state vectors
Cl_idx = 0
Na_idx = 1
H_idx = 2
K_idx = 3

number_of_ions = 4
T = 1000 # total time (in seconds)
dt = 0.001 # duration of integration window (in seconds). Has to be 0.001 or less to make integration stable

RT = 2578.5871 # gas constant x temperature in Kelvin
F = 96485.0 # Faraday constant
c_spec = 0.01  # membrane capacitance per surface area F/(m**2)
buffer_capacity_t0=5.0*1e-4 # initial buffer capacity
U0 = 40*1e-3 # membrane potential in V 

r=1.3e-6 # radius of the vesicle in m
V0 = (4.0/3.0) * np.pi * (r**3) # Initial volume of the vesicle in m3
A0 = 4.0 * np.pi * (r**2) # initial surface area of the vesicle in m2
A_from_V_const=(36.0*np.pi)**(1/3) # constant necessary for calculation surface area from the volume

C0 = A0 * c_spec # inititial membrane capacitance

Cl_o_concentration = 20*1e-3 # external Cl concentraion in M

Na_o_concentration=10*1e-3 # external Na concentraion in M
Na_i_concentration=150*1e-3 # internal Na concentraion in M

K_o_concentration=140*1e-3 # external Cl concentraion in M
K_i_concentration=5*1e-3 # internal K concentraion in M

pH_o =  7.2 # external pH
pH_i =  7.4 # internal pH

hfree_o_concentration = 10**(-pH_o) # concentration of free external protons in M
hfree_i_concentration = 10**(-pH_i) # concentration of free internal protons in M

htotal_o_concentration = hfree_o_concentration/ buffer_capacity_t0 # concentration of total external protons in M
htotal_i_concentration = hfree_i_concentration/ buffer_capacity_t0 # concentration of total intenral protons in M

htotal_i_amount = htotal_i_concentration*V0*1000 # amount of total internal protons in moles

def initialize_internal_concentrations(Cl_i_concentration = 159*1e-3):
   
    # Cl_i_concentration= 159*1e-3 # internal Cl concentraion in M
    # Cl_i_concentration=1*1e-3 # absent internal Cl concentration in M
    # Cl_i_concentration= 9*1e-3 # Cl replacement condition (internal Cl concentration in M)

    Q0=U0*C0 # initial total charge
    X_amount=(Q0/F)-((Na_i_concentration+K_i_concentration+htotal_i_concentration-Cl_i_concentration)*V0*1000) # initial amount of unaccouted ions in moles
    X_concentration=X_amount/(V0*1000) # initial concentration of unaccounted ions in moles

    internal_ions_amounts=[Cl_i_concentration*V0*1000,  Na_i_concentration*V0*1000, htotal_i_concentration*V0*1000, K_i_concentration*V0*1000] # vector of amounts of ions in moles
    external_ions_concentrations = [Cl_o_concentration, Na_o_concentration, htotal_o_concentration, K_o_concentration] # vector of concentrations of external ions
    internal_ions_concentrations = [Cl_i_concentration, Na_i_concentration, htotal_i_concentration, K_i_concentration] # vector of concentrations of internal ions

    Sum_initial_amounts = internal_ions_amounts[Cl_idx] + internal_ions_amounts[Na_idx] + abs(X_amount) + internal_ions_amounts[K_idx] # sum of amounts of all ions

    return X_amount, external_ions_concentrations, internal_ions_amounts, internal_ions_concentrations, Sum_initial_amounts

In [11]:
import argparse

G_asor = 8*1e-5
G_tpc = 2*1e-6
G_k = 0.0
G_CLC = 10*1e-8
G_NHE = 0.0
G_VATPase = 8*1e-9
G_H_leak = 16*1e-9

def parse_user_input():
    """
    Wrapper for arg-parser used to get the input random seed and save mode
    from the user and set defaults if none is provided
    """

    parser = argparse.ArgumentParser()

    parser.add_argument('--gasor', '-as', type = float,
                help = "Conductance of ASOR channel",
                dest = "ASOR", default=G_asor)
    parser.add_argument('--gtpc', '-tpc', type = float,
                help = "Conductance of TPC channel",
                dest = "TPC", default=G_tpc)
    parser.add_argument('--gk', '-k', type = float,
                help = "Conductance of K channel",
                dest = "K", default=G_k)
    parser.add_argument('--gclc', '-cl', type = float,
                help = "Conductance of CLC channel",
                dest = "CLC", default=G_CLC)
    parser.add_argument('--gnhe', '-nhe', type = float,
                help = "Conductance of NHE channel",
                dest = "NHE", default=G_NHE)
    parser.add_argument('--gvatpase', '-atp', type = float,
                help = "Conductance of v_ATPase channel",
                dest = "vATPase", default=G_VATPase)
    parser.add_argument('--ghleak', '-hlk', type = float,
                help = "Conductance of H_leak channel",
                dest = "H_leak", default=G_H_leak)
    parser.add_argument("--ASOR_wt_vs_mutant_vs_none", "-wtmt", type = str,
                help = "Type of ASOR channel: wildtype (wt) vs. mutant (mt)", choices=['wt', 'mt', 'none'],
                dest = "ASOR_pH_dep", default="wt")
    parser.add_argument("--ASOR_U_dep", "-audep", type = str,
                help = "ASOR voltage dependency: wildtype (yes) vs. absent (no)", choices=['yes', 'no'],
                dest = "ASOR_U_dep", default="yes")
    parser.add_argument("--CLC_dep", "-cdep", type = str,
                help = "CLC voltage and pH dependency: present (yes) vs. absent (no)", choices=['yes', 'no'],
                dest = "CLC_dep", default="yes")
    parser.add_argument("--Cli_concentration", "-cli", type = str,
                help = "Initial internal cloride concentration high, low, zero",
                choices = ['high', 'low', 'zero'],
                dest = "Cl_i", default="high")

    arg_dictionary = parser.parse_args([])

    G = {}
    G['ASOR'] = arg_dictionary.ASOR
    G['TPC'] = arg_dictionary.TPC
    G['K'] = arg_dictionary.K
    G['CLC'] = arg_dictionary.CLC
    G['NHE'] = arg_dictionary.NHE
    G['vATPase'] = arg_dictionary.vATPase
    G['H_leak'] = arg_dictionary.H_leak

    ASOR_args = {}
    if arg_dictionary.ASOR_pH_dep == 'wt':
        ASOR_args['ASOR_pH_k2'] = 3.0
        ASOR_args['ASOR_pH_half'] = 5.4
    elif arg_dictionary.ASOR_pH_dep == 'mt':
        ASOR_args['ASOR_pH_k2'] = 1.0
        ASOR_args['ASOR_pH_half'] = 7.4
    elif arg_dictionary.ASOR_pH_dep == 'none':
        ASOR_args['ASOR_pH_k2'] = 0.0
        ASOR_args['ASOR_pH_half'] = 0.0

    if arg_dictionary.ASOR_U_dep == 'yes':
        ASOR_args['ASOR_U_k2'] = 80.0
        ASOR_args['ASOR_U_half'] = -40*1e-3
    elif arg_dictionary.ASOR_U_dep == 'no':
        ASOR_args['ASOR_U_k2'] = 0.0
        ASOR_args['ASOR_U_half'] = 0.0

    CLC_args = {}
    if arg_dictionary.CLC_dep == 'yes':
        CLC_args['CLC_pH_k2'] = 1.5
        CLC_args['CLC_pH_half'] = 5.5
        CLC_args['CLC_U_k2'] = 80.0
        CLC_args['CLC_U_half'] = -40*1e-3
    elif arg_dictionary.CLC_dep == 'no':
        CLC_args['CLC_pH_k2'] = 0
        CLC_args['CLC_pH_half'] = 0
        CLC_args['CLC_U_k2'] = 0
        CLC_args['CLC_U_half'] = 0

    if arg_dictionary.Cl_i == 'high':
        Cl_i_concentration = 159*1e-3
    elif arg_dictionary.Cl_i == 'low':
        Cl_i_concentration = 9*1e-3
    elif arg_dictionary.Cl_i == 'zero':
        Cl_i_concentration = 1*1e-3

    return G, ASOR_args, CLC_args, Cl_i_concentration

## Actual code execution

In [12]:
G, ASOR_args, CLC_args, Cl_i_concentration = parse_user_input()

In [13]:
ASOR_args

{'ASOR_pH_k2': 3.0,
 'ASOR_pH_half': 5.4,
 'ASOR_U_k2': 80.0,
 'ASOR_U_half': -0.04}

In [180]:
X_amount, external_ions_concentrations, internal_ions_amounts, internal_ions_concentrations, Sum_initial_amounts = initialize_internal_concentrations(Cl_i_concentration)

In [181]:
parameters = {
    'dt': dt,
    'T': T,
    'G': G,
    'external_ions_concentrations': external_ions_concentrations,
    'A_from_V_const': A_from_V_const,
    'X_amount': X_amount,
    'buffer_capacity_t0': buffer_capacity_t0, 
    'V_t0': V0, 
    'c_spec': c_spec, 
    'RT': 2578.5871, 
    'F': 96485.0,
    'pH_i': pH_i,
    'U0': U0,
    'A0': A0,
    'C0': C0,
    'Sum_initial_amounts': Sum_initial_amounts,
    'ASOR_pH_k2': ASOR_args['ASOR_pH_k2'],
    'ASOR_pH_half': ASOR_args['ASOR_pH_half'],
    'ASOR_U_k2': ASOR_args['ASOR_U_k2'],
    'ASOR_U_half': ASOR_args['ASOR_U_half'],
    'CLC_pH_k2': CLC_args['CLC_pH_k2'],
    'CLC_pH_half': CLC_args['CLC_pH_half'],
    'CLC_U_k2': CLC_args['CLC_U_k2'],
    'CLC_U_half': CLC_args['CLC_U_half']
   }

In [255]:
asor_config = ASORChannelConfig()

In [250]:
sim = Simulation()

In [252]:
sim.vesicle.__dict__

{'config': <__main__.VesicleConfig at 0x7f5234478110>,
 'init_volume': 9.202772079915702e-18,
 'init_area': 2.1237166338267002e-11,
 'init_capacitance': 2.1237166338267003e-13,
 'init_charge': 8.494866535306801e-15,
 'cl_concentration': 0.159,
 'na_concentration': 0.15,
 'k_concentration': 0.005,
 'pH': 7.4,
 'free_h_concentration': 3.981071705534969e-08,
 'total_h_concentration': 7.962143411069938e-05,
 'unacc_ion_amount': 3.61663937993963e-17}

In [238]:
sim.exterior.total_h_concentration

0.00012619146889603859

In [230]:
vesicle = Vesicle()

In [222]:
vesicle.init_volume

9.202772079915702e-18

In [215]:
import math
math.pi

3.141592653589793

In [182]:
parameters['T'] = 1000

start_time = time.time()
results = run_simulation(internal_ions_amounts, parameters) 
end_time = time.time()
print(f'Time elapsed: {end_time - start_time}')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 999999/999999 [00:18<00:00, 53840.36it/s]

Time elapsed: 18.58290410041809


In [183]:
time_axis = np.arange(0, T, dt)

t = 1

ions_t = np.empty((4, len(time_axis)))
ions_t[:, 0] = internal_ions_amounts

V_t = np.empty(len(time_axis))
V_t[0] = parameters["V_t0"]

G = parameters["G"]
ext_ion = parameters["external_ions_concentrations"]
A_from_V_const = parameters["A_from_V_const"]
X_amount = parameters["X_amount"]
buffer_capacity_t0 = parameters["buffer_capacity_t0"]
V_t0 = parameters["V_t0"]
c_spec = parameters["c_spec"]
RT = parameters["RT"]
F = parameters["F"]

dIons_dt, fluxes, deps, local_vars = compute_flows(
            time_axis[t],
            ions_t[:, t - 1],
            V_t[t - 1],
            G,
            ext_ion,
            A_from_V_const,
            X_amount,
            buffer_capacity_t0,
            V_t0,
            c_spec,
            parameters["ASOR_pH_k2"],
            parameters["ASOR_pH_half"],
            parameters["ASOR_U_k2"],
            parameters["ASOR_U_half"],
            parameters["CLC_pH_k2"],
            parameters["CLC_pH_half"],
            parameters["CLC_U_k2"],
            parameters["CLC_U_half"],
        )

In [192]:
def masha_compute_flows(
    time_step,
    internal_ions,
    V,
    G,
    external_ions_concentrations,
    A_from_V_const,
    X_amount,
    buffer_capacity_T0,
    V_t0,
    c_spec,
    ASOR_pH_k2,
    ASOR_pH_half,
    ASOR_U_k2,
    ASOR_U_half,
    CLC_pH_k2,
    CLC_pH_half,
    CLC_U_k2,
    CLC_U_half,
    RT=2578.5871,
    F=96485.0,
):
    """
    Arguments
    =============
    `time_step` [float]:
        - which timestep (in seconds)
    `internal_ions` [1D np.ndarray]:
        -vector of internal ion amounts (in moles)
    `external_ions_concentrations` [1D np.ndarray]:
        -vector of external ion concentrations (in moles/l)
    `V_t` [float]:
        - current volume (in m3)
    "G" [dict]:
        - dictionary containing conductances (in UNITS HERE) for the following channel types:
            - ASOR, ClC, TPC, NHE, vATPase, H leak, K channel
    `X_amount` [float]:
        - amount of unaccounted for ions (in moles)
    `A_from_V_const` [float]
        - area calculated from the volume
    `buffer_capacity_T0` [float]:
        - buffer capacity at initial time
    `V_t0` [float]:
        - initial volume (in m3)
    `c_spec` [float]:
        - membrane capacitance (in F/(m**2))
    `RTdivF` [float]:
        - RT / F
    `ASOR_pH_k2` [float]:
        - exponential scaling parameter of the ASOR pH dependence
    `ASOR_pH_half` [float]:
        - pH1/2 of ASOR activation
    `ASOR_U_k2` [float]:
        - exponential scaling parameter of the ASOR voltage dependence
    `ASOR_U_half` [float]:
        - U1/2 of ASOR activation
    `CLC_pH_k2` [float]:
        - exponential scaling parameter of the CLC pH dependence
    `CLC_pH_half` [float]:
        - pH1/2 of CLC activation
    `CLC_U_k2` [float]:
        - exponential scaling parameter of the CLC voltage dependence
    `CLC_U_half` [float]:
        - U1/2 of CLC activation
    `RT` [float]:
        - Gas constant x Temperature
    `F` [float]:
        - Faraday's constant

    Returns
    ===========
    `dIons_dt` [1D np.ndarray]:
        - vector of fluxes for Cl, Na, H and K ions
    `fluxes` [dict]:
        - dictionary containing fluxes of each ion through each ion chanel
    `deps` [dict]:
        - dictionary containing pH-, time- and V- dependency functions for ASOR, ClC and vATPase
    `local_vars` [dict]:
        - dictionary containing other macropinosme parameters: pH, membrane area, capacitance  and buffer capacity

    """

    dIons_dt = np.empty(internal_ions.shape[0])

    Buffer_T = buffer_capacity_T0 * (
        V / V_t0
    )  # Buffer capacity at the current time step
    htotal_i = internal_ions[H_idx] / (
        V * 1000
    )  # Concentration (mol/l) of H (total) in MP from the amount (mol)

    hfree_o = (
        external_ions_concentrations[H_idx] * buffer_capacity_T0
    )  # Free external H concentration (mol/l)
    hfree_i = htotal_i * Buffer_T  # Free internal H concentration (mol/l)

    A = A_from_V_const * (
        V ** (2 / 3)
    )  # Membrane area of the macropinosome at the current time step
    C = A * c_spec  # Membrane capacitance
    Q = (
        internal_ions[Na_idx]
        + internal_ions[K_idx]
        + internal_ions[H_idx]
        - internal_ions[Cl_idx]
        + X_amount
    ) * F  # Total charge
    U = Q / C  # Membrane potential

    cl_i = internal_ions[Cl_idx] / (V * 1000)  # Internal Cl concentration in mol/l
    na_i = internal_ions[Na_idx] / (V * 1000)  # Internal Na concentration in mol/l
    k_i = internal_ions[K_idx] / (V * 1000)  # Internal K concentration in mol/l

    pH_local = -np.log10(hfree_i)  # pH inside of the macropinosome

    RTdivF = RT / F

    #Nernst potentials computed for different channels
    # potential_asor = P.nernst_potential_Cl_asor(
    #     U, external_ions_concentrations[Cl_idx], cl_i, RTdivF
    # )
    # potential_CLC = P.nernst_potential_CLC(
    #     U, cl_i, external_ions_concentrations[Cl_idx], hfree_i, hfree_o, RT, F
    # )
    # n_potential_Na_TPC = P.nernst_potential_Na_K(
    #     U, external_ions_concentrations[Na_idx], na_i, RTdivF
    # )
    # potential_nhe = P.potential_NHE(
    #     external_ions_concentrations[Na_idx], na_i, hfree_i, hfree_o
    # )
    # potential_k = P.nernst_potential_Na_K(
    #     U, external_ions_concentrations[K_idx], k_i, RTdivF
    # )
    # potential_VATPase = P.nernst_potential_VATPase(U, hfree_o, hfree_i, RTdivF)
    # potential_H_leak = P.nernst_potential_H_leak(U, hfree_o, hfree_i, RTdivF)

    # # print(potential_asor, G['ASOR'], U, pH_local, A, ASOR_pH_k2, ASOR_pH_half, ASOR_U_k2, ASOR_U_half)
    # # print("Type of potential_asor:", type(potential_asor))
    # # print("Type of G['ASOR']:", type(G['ASOR']))
    # # print("Type of U:", type(U))
    # # print("Type of pH_local:", type(pH_local))
    # # print("Type of A:", type(A))
    # # print("Type of ASOR_pH_k2:", type(ASOR_pH_k2))
    # # print("Type of ASOR_pH_half:", type(ASOR_pH_half))
    # # print("Type of ASOR_U_k2:", type(ASOR_U_k2))
    # # print("Type of ASOR_U_half:", type(ASOR_U_half))
    # # quit()

    # # Flow of chloride (d[Cl]/dt)
    # Cl_flux_asor = i_flux.J_cl_asor(
    #     potential_asor,
    #     G["ASOR"],
    #     U,
    #     pH_local,
    #     A,
    #     ASOR_pH_k2,
    #     ASOR_pH_half,
    #     ASOR_U_k2,
    #     ASOR_U_half,
    # )
    # Cl_flux_CLC = i_flux.J_Cl_CLC(
    #     potential_CLC,
    #     G["CLC"],
    #     U,
    #     pH_local,
    #     A,
    #     CLC_pH_k2,
    #     CLC_pH_half,
    #     CLC_U_k2,
    #     CLC_U_half,
    # )

    # dIons_dt[Cl_idx] = Cl_flux_asor + Cl_flux_CLC

    # # Flow of sodium (d[Na]/dt)
    # na_flux_tpc = i_flux.J_na_tpc(n_potential_Na_TPC, G["TPC"], A)
    # na_flux_nhe = i_flux.J_Na_NHE(potential_nhe, G["NHE"], A)

    # dIons_dt[Na_idx] = na_flux_tpc + na_flux_nhe

    # # Flow of protons (d[H]/dt)
    # H_flux_CLC = i_flux.J_H_CLC(
    #     potential_CLC,
    #     G["CLC"],
    #     U,
    #     pH_local,
    #     A,
    #     CLC_pH_k2,
    #     CLC_pH_half,
    #     CLC_U_k2,
    #     CLC_U_half,
    # )
    # H_flux_NHE = i_flux.J_H_NHE(potential_nhe, G["NHE"], A)
    # H_flux_VATPase = i_flux.J_VATPase(potential_VATPase, G["vATPase"], time_step, A)
    # H_flux_leak = i_flux.J_H_leak(potential_H_leak, G["H_leak"], A)

    # dIons_dt[H_idx] = H_flux_CLC + H_flux_NHE + H_flux_VATPase + H_flux_leak

    # # calculate flow of K (d[K]/dt)
    # K_flux = i_flux.J_k(potential_k, G["K"], A)

    # dIons_dt[K_idx] = K_flux

    # pH_dep_ASOR = dep_funct.pH_dependence_ASOR(
    #     pH_local, pH_k2=ASOR_pH_k2, pH_half=ASOR_pH_half
    # )
    # v_dep_ASOR = dep_funct.v_dependence_ASOR(U, U_k2=ASOR_U_k2, U_half=ASOR_U_half)

    # pH_dep_ClC = dep_funct.pH_dependence_ClC(
    #     pH_local, pH_k2=CLC_pH_k2, pH_half=CLC_pH_half
    # )
    # v_dep_ClC = dep_funct.v_dependence_ClC(U, U_k2=CLC_U_k2, U_half=CLC_U_half)

    # t_dep_VATPase = dep_funct.g_VATP_dependence(time_step)

    # fluxes = {
    #     "Cl_asor": Cl_flux_asor,
    #     "Cl_CLC": Cl_flux_CLC,
    #     "Na_tpc": na_flux_tpc,
    #     "Na_nhe": na_flux_nhe,
    #     "H_CLC": H_flux_CLC,
    #     "H_NHE": H_flux_NHE,
    #     "H_VATPase": H_flux_VATPase,
    #     "H_leak": H_flux_leak,
    #     "K": K_flux,
    # }

    # deps = {
    #     "pH_ASOR": pH_dep_ASOR,
    #     "v_ASOR": v_dep_ASOR,
    #     "pH_ClC": pH_dep_ClC,
    #     "v_ClC": v_dep_ClC,
    #     "t_VATPase": t_dep_VATPase,
    # }

    # local_vars = {"pH_local": pH_local, "A": A, "C": C, "Buffer_T": Buffer_T, "U": U}

    # return dIons_dt, fluxes, deps, local_vars

In [193]:
def masha_flows():
    masha_compute_flows(
            time_axis[t],
            ions_t[:, t - 1],
            V_t[t - 1],
            G,
            ext_ion,
            A_from_V_const,
            X_amount,
            buffer_capacity_t0,
            V_t0,
            c_spec,
            parameters["ASOR_pH_k2"],
            parameters["ASOR_pH_half"],
            parameters["ASOR_U_k2"],
            parameters["ASOR_U_half"],
            parameters["CLC_pH_k2"],
            parameters["CLC_pH_half"],
            parameters["CLC_U_k2"],
            parameters["CLC_U_half"],
        )
    pass

In [194]:
%timeit masha_flows()

1.79 µs ± 41.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [119]:
from math import log10

In [188]:
cl_idx = 0
na_idx = 1
h_idx = 2
k_idx = 3

n_ions = 4
init_buffer_capacity = buffer_capacity_t0
init_volume = V_t0

volume2buffer_capacity_const = init_buffer_capacity / init_volume

free_ext_h_concentration = ext_ion[h_idx] * init_buffer_capacity

volume2area_const = (36.0 * np.pi) ** (1 / 3)

ion_charges = np.asarray([-1, 1, 1, 1], dtype=np.float64)

nernst_constant = RT / F

def my_compute_flows(time: float = None,
             n_ions: int = None,
             int_ion_amounts: np.ndarray = None,
             volume: float = None,
             #channel_conductances: dict[str, float] = None,
             ext_ion_concentrations: np.ndarray = None,
             volume2area_const: float = None,
             int_unacc_ion_amount: float = None,
             init_buffer_capacity: float = None,
             init_volume: float = None,
             volume2buffer_capacity_const: float = None,
             free_ext_h_concentration: float = free_ext_h_concentration,
             specific_capacitance: float = None,             
             ASOR_pH_k2: float = None,
             ASOR_pH_half: float = None,
             ASOR_U_k2: float = None,
             ASOR_U_half: float = None,
             CLC_pH_k2: float = None,
             CLC_pH_half: float = None,
             CLC_U_k2: float = None,
             CLC_U_half: float = None,
             RT: float = 2578.5871,
             faraday_constant: float = 96485.0):
    #int_ion_amount_grad = np.zeros(n_ions, dtype=np.float64)
    buffer_capacity = volume2buffer_capacity_const * volume

    amount2concentration_const = 1 / (volume * 1000)

    area = volume2area_const * (volume ** (2/3))
    capacitance = area * specific_capacitance

    total_charge = -int_ion_amounts[0] + int_ion_amounts[1] + int_ion_amounts[2] + int_ion_amounts[3] + int_unacc_ion_amount 

    voltage = total_charge * capacitance

    int_cl_concentration = int_ion_amounts[cl_idx] * amount2concentration_const
    int_na_concentration = int_ion_amounts[na_idx] * amount2concentration_const
    int_h_concentration = int_ion_amounts[h_idx] * amount2concentration_const
    int_k_concentration = int_ion_amounts[k_idx] * amount2concentration_const

    free_int_h_concentration = int_h_concentration * buffer_capacity

    local_pH = -log10(free_int_h_concentration)    

In [189]:
from numba import jit

In [190]:
def my_flows():
    my_compute_flows(time=time_axis[t],
                     n_ions=n_ions,
                     int_ion_amounts=ions_t[:, t - 1],
                     volume=V_t[t - 1],
                     #channel_conductances=G,
                     ext_ion_concentrations=ext_ion,
                     volume2area_const=A_from_V_const,
                     int_unacc_ion_amount=X_amount,
                     init_buffer_capacity=buffer_capacity_t0,
                     init_volume=V_t0,
                     volume2buffer_capacity_const=volume2buffer_capacity_const,
                     free_ext_h_concentration=free_ext_h_concentration,
                     specific_capacitance=c_spec,
                     ASOR_pH_k2=parameters["ASOR_pH_k2"],
                     ASOR_pH_half=parameters["ASOR_pH_half"],
                     ASOR_U_k2=parameters["ASOR_U_k2"],
                     ASOR_U_half=parameters["ASOR_U_half"],
                     CLC_pH_k2=parameters["CLC_pH_k2"],
                     CLC_pH_half=parameters["CLC_pH_half"],
                     CLC_U_k2=parameters["CLC_U_k2"],
                     CLC_U_half=parameters["CLC_U_half"],
        )
    pass

In [191]:
%timeit my_flows()

1.61 µs ± 6.51 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import backend

In [3]:
from backend.simulation import Simulation

In [4]:
sim = Simulation()

In [6]:
sim.make_snapshot()

{'exterior': {'cl_concentration': 0.02,
  'na_concentration': 0.01,
  'k_concentration': 0.14,
  'pH': 7.2,
  'free_h_concentration': 6.30957344480193e-08,
  'total_h_concentration': 0.00012619146889603859},
 'vesicle': {'cl_conc': 0.159,
  'na_conc': 0.15,
  'k_conc': 0.005,
  'pH': 7.4,
  'free_h_conc': 3.981071705534969e-08,
  'total_h_conc': 7.962143411069938e-05,
  'unacc_ion_amount': 3.61663937993963e-17,
  'unacc_ion_conc': 0.003929945616965403}}

In [6]:
sim.vesicle.__dict__

{'config': <backend.vesicle.VesicleConfig at 0x7f968c27daf0>,
 'init_volume': 9.202772079915702e-18,
 'init_area': 2.1237166338267002e-11,
 'init_capacitance': 2.1237166338267003e-13,
 'init_charge': 8.494866535306801e-15,
 'cl_conc': 0.159,
 'na_conc': 0.15,
 'k_conc': 0.005,
 'pH': 7.4,
 'free_h_conc': 3.981071705534969e-08,
 'total_h_conc': 7.962143411069938e-05,
 'unacc_ion_amount': 3.61663937993963e-17,
 'unacc_ion_conc': 0.003929945616965403}